# Pluggable TTA Implementation

## Import Libraries

In [ ]:
%pip install torchinfo gdown

In [ ]:
%pip install git+https://github.com/robustaim/YOLOV

In [1]:
import os
from os import path

import torch
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset

from torchvision import datasets, transforms
from torchvision.ops import complete_box_iou, box_iou
from yolox.models.yolox import YOLOX
from yolox.models.yolo_head import YOLOXHead

import numpy as np
import pandas as pd

from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import pygwalker as pyg
import wandb

datasets.utils.tqdm = tqdm

/home/lab211/Desktop/P-TTA/.venv/lib/python3.11/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


### Check GPU Availability

In [ ]:
# WandB Initialization
wandb.init(project="yolox_apt")

In [ ]:
!nvidia-smi

In [ ]:
# Set CUDA Device Number 0~7
DEVICE_NUM = 0
ADDITIONAL_GPU = 0

if torch.cuda.is_available():
    if ADDITIONAL_GPU:
        torch.cuda.set_device(DEVICE_NUM)
        device = torch.device("cuda")
    else:
        device = torch.device(f"cuda:{DEVICE_NUM}")
else:
    device = torch.device("cpu")
    DEVICE_NUM = -1

print(f"INFO: Using device - {device}" + (f":{DEVICE_NUM}" if ADDITIONAL_GPU else ""))

### GOT-10k Dataset for Next-frame Prediction Task (Default Pretraining Process)
http://got-10k.aitestunion.com/downloads

#### Data File Structure
The downloaded and extracted full dataset should follow the file structure:
```
    |-- GOT-10k/
        |-- train/
        |  |-- GOT-10k_Train_000001/
        |  |   ......
        |  |-- GOT-10k_Train_009335/
        |  |-- list.txt
        |-- val/
        |  |-- GOT-10k_Val_000001/
        |  |   ......
        |  |-- GOT-10k_Val_000180/
        |  |-- list.txt
        |-- test/
        |  |-- GOT-10k_Test_000001/
        |  |   ......
        |  |-- GOT-10k_Test_000180/
        |  |-- list.txt
```

#### Annotation Description
Each sequence folder contains 4 annotation files and 1 meta file. A brief description of these files follows (let N denotes sequence length):

* groundtruth.txt -- An N×4 matrix with each line representing object location [xmin, ymin, width, height] in one frame.
* cover.label -- An N×1 array representing object visible ratios, with levels ranging from 0~8.
* absense.label -- An binary N×1 array indicating whether an object is absent or present in each frame.
* cut_by_image.label -- An binary N×1 array indicating whether an object is cut by image in each frame.
* meta_info.ini -- Meta information about the sequence, including object and motion classes, video URL and more.
* Values 0~8 in file cover.label correspond to ranges of object visible ratios: 0%, (0%, 15%], (15%~30%], (30%, 45%], (45%, 60%], (60%, 75%], (75%, 90%], (90%, 100%) and 100% respectively.

## Load Dataset

In [ ]:
from apt.datasets import GOT10kDataset

In [ ]:
# Define image size for resizing
IMG_SIZE = 576

# Define image normalization parameters
IMG_NORM = {
    'mean': [0.485, 0.456, 0.406],
    'std': [0.229, 0.224, 0.225]
}

# Create transforms
train_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),  # Adjust brightness/contrast
    transforms.ToTensor(),
    transforms.Normalize(**IMG_NORM)
])

test_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(), 
    transforms.Normalize(**IMG_NORM)
])

In [ ]:
DATA_ROOT = path.join(".", "data")

train_dataset = GOT10kDataset(root=DATA_ROOT, force_download=False, train=True, transform=train_transform)
valid_dataset = GOT10kDataset(root=DATA_ROOT, force_download=False, valid=True, transform=train_transform)
test_dataset = GOT10kDataset(root=DATA_ROOT, force_download=False, train=False, transform=test_transform)

print(f"INFO: Dataset loaded successfully. Number of samples - Train({len(train_dataset)}), Valid({len(valid_dataset)}), Test({len(test_dataset)})")

In [ ]:
# Train Dataset Distribution
pyg.walk(train_dataset.df)

In [ ]:
train_dataset

In [ ]:
from apt.datasets import PairedGOT10kDataset

In [ ]:
# Create paired datasets with lazy loading
train_pairset, valid_pairset = PairedGOT10kDataset.create_train_val_split(train_dataset)
test_pairset = PairedGOT10kDataset(base_dataset=valid_dataset)

print(f"INFO: PairedDataset initialized. Total sequences - Train({len(train_pairset)}), Valid({len(valid_pairset)}), Test({len(test_pairset)})")

In [ ]:
train_pairset[0]

In [ ]:
def visualize_frame_pair(pairset, idx=None, figsize=(15, 7)):
    """
    Visualize a pair of consecutive frames with their bounding boxes.
    
    Args:
        pairset: PairedGOT10kDataset instance
        idx: Index of the pair to visualize. If None, picks a random index
        figsize: Size of the figure as (width, height)
    """
    # Get random index if not provided
    if idx is None:
        idx = np.random.randint(len(pairset))
    
    # Get frame pair
    prev_img, curr_img, prev_gt, curr_gt = pairset[idx]
    
    def draw_bbox(bbox, color='red'):
        """Helper function to draw bounding box"""
        plt.plot([bbox[0], bbox[0]+bbox[2]], [bbox[1], bbox[1]], color=color, linewidth=2)
        plt.plot([bbox[0], bbox[0]], [bbox[1], bbox[1]+bbox[3]], color=color, linewidth=2)
        plt.plot([bbox[0]+bbox[2], bbox[0]+bbox[2]], [bbox[1], bbox[1]+bbox[3]], color=color, linewidth=2)
        plt.plot([bbox[0], bbox[0]+bbox[2]], [bbox[1]+bbox[3], bbox[1]+bbox[3]], color=color, linewidth=2)

    # Create figure
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=figsize)

    # Plot previous frame
    plt.subplot(1, 2, 1)
    plt.imshow(prev_img)
    draw_bbox(prev_gt.numpy())
    plt.title('Previous Frame')
    
    # Plot current frame
    plt.subplot(1, 2, 2)
    plt.imshow(curr_img)
    draw_bbox(curr_gt.numpy())
    plt.title('Current Frame')
    
    plt.tight_layout()
    plt.show()
    
    return idx

In [ ]:
def visualize_normalized_frame_pair(pairset, idx=None, imgsize=(IMG_SIZE, IMG_SIZE), figsize=(7, 5)):
    """
    Visualize a pair of consecutive frames with their bounding boxes.
    
    Args:
        pairset: PairedGOT10kDataset instance
        idx: Index of the pair to visualize. If None, picks a random index
        imgsize: Size of the image as (width, height)
        figsize: Size of the figure as (width, height)
    """
    # Get random index if not provided
    if idx is None:
        idx = np.random.randint(len(pairset))
    
    # Get frame pair
    prev_img, curr_img, prev_gt, curr_gt = pairset[idx]
    
    # Convert tensors to numpy arrays and denormalize
    def denormalize(img_tensor):
        # Move channels to last dimension
        img = img_tensor.permute(1, 2, 0).numpy()
        # Denormalize
        img = img * np.array(IMG_NORM['std']) + np.array(IMG_NORM['mean'])
        # Clip values to valid range
        img = np.clip(img, 0, 1)
        return img
    
    prev_img = denormalize(prev_img)
    curr_img = denormalize(curr_img)
    
    def draw_bbox(ax, bbox, color='red'):
        """Helper function to draw bounding box"""
        x, y, w, h = bbox.numpy()
        x *= imgsize[0]
        y *= imgsize[1]
        w *= imgsize[0]
        h *= imgsize[1]
        ax.plot([x-w/2, x+w/2], [y-h/2, y-h/2], color=color, linewidth=2)
        ax.plot([x-w/2, x-w/2], [y-h/2, y+h/2], color=color, linewidth=2)
        ax.plot([x+w/2, x+w/2], [y-h/2, y+h/2], color=color, linewidth=2)
        ax.plot([x-w/2, x+w/2], [y+h/2, y+h/2], color=color, linewidth=2)

    # Create figure
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=figsize)
    
    # Plot previous frame
    ax1.imshow(prev_img)
    draw_bbox(ax1, prev_gt)
    ax1.set_title('Previous Frame')
    ax1.axis('off')
    
    # Plot current frame
    ax2.imshow(curr_img)
    draw_bbox(ax2, curr_gt)
    ax2.set_title('Current Frame')
    ax2.axis('off')
    
    plt.tight_layout()
    plt.show()
    
    return idx

In [ ]:
for _ in range(2):
    selected_idx = visualize_normalized_frame_pair(train_pairset)
    print(f"Visualized pair index: {selected_idx}")

## DataLoader

In [ ]:
# Set Batch Size
BATCH_SIZE = 256, 256, 1

In [ ]:
# Use Teacher Forcing
train_pairset.use_teacher_forcing = True
valid_pairset.use_teacher_forcing = True

In [ ]:
MULTI_PROCESSING = True  # Set False if DataLoader is causing issues

import multiprocessing
from platform import system

cpu_cores = multiprocessing.cpu_count()
num_workers = 12
if MULTI_PROCESSING and system() != "Windows":  # Multiprocess data loading is not supported on Windows
    num_workers += ADDITIONAL_GPU
    print(f"INFO: Number of CPU cores - {num_workers}/{cpu_cores}")
else:
    print("INFO: Using DataLoader without multi-processing.")

train_loader = DataLoader(train_pairset, batch_size=BATCH_SIZE[0], shuffle=True, num_workers=num_workers)
valid_loader = DataLoader(valid_pairset, batch_size=BATCH_SIZE[1], shuffle=False, num_workers=num_workers)
test_loader = DataLoader(test_pairset, batch_size=BATCH_SIZE[2], shuffle=False, num_workers=num_workers)

## Define Model
### APT: Adaptive Plugin for TTA (Test-time Adaptation)

In [ ]:
yolo_pretrained = YOLOX(head=YOLOXHead(30))
yolo_pretrained.load_state_dict(torch.load("./pretrained/yoloxl_vid.pth", map_location="cpu")['model'])

for param in yolo_pretrained.parameters():
    param.requires_grad = False  # Freeze YOLO

yolo_pretrained

In [ ]:
class FeatureNormalizationLayer(nn.Module):
    def __init__(self, target_dim=256):
        super().__init__()
        self.target_dim = target_dim
        
        # Keep only channel dimension
        self.adaptive_pool = nn.AdaptiveAvgPool2d((1, 1))
        
        # Linear compression
        self.linear_compress = nn.AdaptiveAvgPool1d(target_dim)
        
        # Feature normalization
        self.feature_norm = nn.Sequential(
            nn.LayerNorm(target_dim),
            nn.Dropout(0.1)
        )

    def forward(self, x):
        # Apply adaptive pooling
        x = self.adaptive_pool(x)
        
        # Squeeze channel dimension
        x = x.squeeze(-1).squeeze(-1)
        
        # Linear compression
        x = self.linear_compress(x)
        
        # Feature normalization
        x = self.feature_norm(x)
        
        return x

In [ ]:
class APT(nn.Module):
    """
    Autoencoder for Adaptation
    which learns how to sniff out the frame changes to predict next bounding boxes.
    """

    def __init__(self, feature_dim=256, bbox_dim=4, hidden_dim=32):
        super().__init__()

        self.feature_dim = feature_dim
        self.bbox_dim = bbox_dim
        self.hidden_dim = hidden_dim

        # Feature normalization layer for encoder-agnostic adaptation
        self.feature_norm = FeatureNormalizationLayer(target_dim=feature_dim)

        # Lightweight feature sniffer
        self.feature_sniffer = nn.Sequential(
            nn.Linear(feature_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim // 4 * 3)
        )

        # Previous bbox encoder
        self.bbox_encoder = nn.Sequential(
            nn.Linear(bbox_dim, hidden_dim // 4),
            nn.ReLU()
        )

        # Fusion layer
        self.fusion = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU()
        )

        # Prediction head
        self.predictor = nn.Sequential(
            nn.Linear(hidden_dim, bbox_dim),
            nn.Sigmoid()  # Normalize bbox coordinates to [0,1]
        )

        # Optional: Sparsity regularization
        self.activation = {}

    def forward(self, features, prev_bbox):
        # Normalize encoder features to be encoder-agnostic
        norm_features = self.feature_norm(features)

        # Extract relevant features from current frame
        sniffed_features = self.feature_sniffer(norm_features)

        # Encode previous bbox information
        bbox_features = self.bbox_encoder(prev_bbox)

        # Fuse features
        fused = self.fusion(
            torch.cat([sniffed_features, bbox_features], dim=-1)
        )

        # Predict next bbox
        next_bbox = self.predictor(fused)

        return next_bbox

In [ ]:
class MultiScaleAPT(nn.Module):
    """
    Autoencoder for Adaptation (Multi-Scale Approach)
    which learns how to sniff out the frame changes to predict next bounding boxes.
    """

    def __init__(self, feature_dim=256, scale_dim=[256, 512, 1024], bbox_dim=4, hidden_dim=512):
        super().__init__()

        self.feature_dim = feature_dim
        self.bbox_dim = bbox_dim
        self.hidden_dim = hidden_dim

        # Feature normalization layer for encoder-agnostic adaptation
        self.feature_norms = nn.ModuleList([
            FeatureNormalizationLayer(dim) for dim in scale_dim
        ])

        # Combiner
        self.projections = nn.ModuleList([
            nn.Linear(dim, feature_dim) for dim in scale_dim
        ])

        # Lightweight feature sniffer
        self.feature_sniffer = nn.Sequential(
            nn.Linear(feature_dim * len(scale_dim), hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim // 4 * 3)
        )

        # Previous bbox encoder
        self.bbox_encoder = nn.Sequential(
            nn.Linear(bbox_dim, hidden_dim // 4),
            nn.ReLU()
        )

        # Fusion layer
        self.fusion = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU()
        )

        # Prediction head
        self.predictor = nn.Sequential(
            nn.Linear(hidden_dim, bbox_dim),
            nn.Sigmoid()  # Normalize bbox coordinates to [0,1]
        )

        # Optional: Sparsity regularization
        self.activation = {}

    def forward(self, features, prev_bbox):
        # Normalize encoder features to be encoder-agnostic
        normalized_features = []
        for i, feature in enumerate(features):
            norm_feat = self.feature_norms[i](feature)
            proj_feat = self.projections[i](norm_feat)
            normalized_features.append(proj_feat)

        # Extract relevant features from current frame
        concat_features = torch.cat(normalized_features, dim=1)
        sniffed_features = self.feature_sniffer(concat_features)

        # Encode previous bbox information
        bbox_features = self.bbox_encoder(prev_bbox)

        # Fuse features
        fused = self.fusion(
            torch.cat([sniffed_features, bbox_features], dim=-1)
        )

        # Predict next bbox
        next_bbox = self.predictor(fused)

        return next_bbox

In [ ]:
class TestTimeAdaptiveYOLOX(nn.Module):
    def __init__(
        self, encoder: nn.Module, decoder: nn.Module,
        feature_dim=256, bbox_dim=4, hidden_dim=32
    ):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.apt = MultiScaleAPT(
            feature_dim=feature_dim, bbox_dim=bbox_dim,
            hidden_dim=hidden_dim
        )

    def forward(self, current_frame, prev_bbox, use_teacher_forcing=False):
        # Extract features using YOLO encoder
        features = self.encoder(current_frame)

        # Adapt using APT
        pred_bbox = self.apt(features, prev_bbox)

        if use_teacher_forcing:
            output = None
        else:
            # Use YOLO decoder for final prediction
            output = self.decoder(features)

        return output, pred_bbox

In [ ]:
# Initialize Model
model = TestTimeAdaptiveYOLOX(
    encoder=yolo_pretrained.backbone, decoder=yolo_pretrained.head,
    feature_dim=256, bbox_dim=4, hidden_dim=512
)

if ADDITIONAL_GPU:
    model = nn.DataParallel(model, device_ids=list(range(DEVICE_NUM, DEVICE_NUM+ADDITIONAL_GPU+1)))
model.to(device)

## Training

In [ ]:
def cxcywh_to_xyxy(boxes):
    """Convert bbox coordinates from (cx, cy, w, h) to (x1, y1, x2, y2)"""
    cx, cy, w, h = boxes.unbind(-1)
    x1 = cx - 0.5 * w
    y1 = cy - 0.5 * h
    x2 = cx + 0.5 * w
    y2 = cy + 0.5 * h
    return torch.stack((x1, y1, x2, y2), dim=-1)

In [ ]:
def compute_confidence_from_iou(pred_boxes, gt_boxes):
    """Compute confidence scores using IoU values"""
    iou_matrix = box_iou(pred_boxes, gt_boxes)  # Compute IoU
    confidence_scores, _ = torch.max(iou_matrix, dim=1)  # Use max IoU as confidence
    return confidence_scores

In [ ]:
def m_ap(gt_boxes, pred_boxes, pred_scores=None, iou_threshold=0.5):
    """Compute Average Precision (AP) using PyTorch"""
    # Compute IoU matrix
    iou_matrix = box_iou(pred_boxes, gt_boxes)

    # Sort predicted boxes by confidence scores (descending order)
    if not pred_scores:
        pred_scores = compute_confidence_from_iou(pred_boxes, gt_boxes)
    sorted_indices = torch.argsort(-pred_scores)
    pred_boxes = pred_boxes[sorted_indices]
    pred_scores = pred_scores[sorted_indices]
    iou_matrix = iou_matrix[sorted_indices]

    # Determine True Positives (TP) and False Positives (FP)
    tp = torch.zeros(len(pred_boxes))
    fp = torch.zeros(len(pred_boxes))
    matched_gt = torch.zeros(len(gt_boxes))  # Track matched ground truth boxes

    for i, ious in enumerate(iou_matrix):
        best_iou, best_gt_idx = torch.max(ious, dim=0)
        if best_iou >= iou_threshold and matched_gt[best_gt_idx] == 0:
            tp[i] = 1  # True Positive
            matched_gt[best_gt_idx] = 1  # Mark ground truth as matched
        else:
            fp[i] = 1  # False Positive

    # Compute Precision-Recall curve
    tp_cumsum = torch.cumsum(tp, dim=0)
    fp_cumsum = torch.cumsum(fp, dim=0)
    recall = tp_cumsum / len(gt_boxes)
    precision = tp_cumsum / (tp_cumsum + fp_cumsum + 1e-6)

    # Compute AP (area under Precision-Recall curve)
    ap = torch.trapz(precision, recall)  # Trapezoidal Rule
    return ap

### Default Pre-training Process
Using Teacher forcing

In [ ]:
# Set Epoch Count & Learning Rate
EPOCHS = 10
LEARNING_RATE = 1e-4, 1e-6
WEIGHT_DECAY = 0.05
MSE_WEIGHT = 0.05
CIoU_WEIGHT = 0.95

criterion = nn.MSELoss()
wandb.watch(model, criterion, log="all", log_freq=10)
optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE[0], weight_decay=WEIGHT_DECAY)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS, eta_min=LEARNING_RATE[1])

In [ ]:
model_id = "apt_model"
if not path.isdir(path.join(".", "models")) or not path.isdir(path.join(".", "models", model_id)):
    os.mkdir(path.join(".", "models"))
    os.mkdir(path.join(".", "models", model_id))

train_length, valid_length = map(len, (train_loader, valid_loader))
epochs = tqdm(range(EPOCHS), desc="Running Epochs")
with (tqdm(total=train_length, desc="Training") as train_progress,
        tqdm(total=valid_length, desc="Validation") as valid_progress):  # Set up Progress Bars
    best_ciou = float("inf")

    for epoch in epochs:
        train_progress.reset(total=train_length)
        valid_progress.reset(total=valid_length)

        train_loss, train_ciou, train_map = 0, 0, 0

        # Training
        model.train()
        for i, (curr_frame, prev_bbox, curr_bbox) in enumerate(train_loader):
            optimizer.zero_grad()

            prev_bbox, curr_bbox = prev_bbox.to(device), curr_bbox.to(device)
            _, pred_bbox = model(curr_frame.to(device), prev_bbox, use_teacher_forcing=True)  # Use Teacher Forcing while training

            pred_boxes_list = [pred_bbox[i].unsqueeze(0) for i in range(len(pred_bbox))]
            target_boxes_list = [curr_bbox[i].unsqueeze(0) for i in range(len(curr_bbox))]

            mse_loss = criterion(pred_bbox, curr_bbox)
            ciou_values = complete_box_iou(cxcywh_to_xyxy(pred_bbox), cxcywh_to_xyxy(curr_bbox))
            ciou_loss = 1 - torch.diag(ciou_values).mean()

            (MSE_WEIGHT*mse_loss/BATCH_SIZE[0] + CIoU_WEIGHT*ciou_loss).backward()
            optimizer.step()
            scheduler.step()

            train_loss += (mse_loss/BATCH_SIZE[0]).item() / train_length
            train_ciou += ciou_loss / train_length
            map_score = m_ap(cxcywh_to_xyxy(pred_bbox), cxcywh_to_xyxy(curr_bbox))
            train_map += map_score / train_length

            train_progress.update(1)
            if i != train_length-1: wandb.log({'MSE Loss': mse_loss.item()/BATCH_SIZE[0], 'CIoU Loss': ciou_loss, 'mAP': map_score})
            print(f"\rEpoch [{epoch+1:2}/{EPOCHS}], Step [{i+1:4}/{train_length}], MSE Loss: {mse_loss.item()/BATCH_SIZE[0]:.6%}, CIoU Loss: {ciou_loss:.6f}, mAP: {map_score:.2f}", end="")

        print(f"\rEpoch [{epoch+1:2}/{EPOCHS}], Step [{train_length}/{train_length}], MSE Loss: {train_loss:.6%}, CIoU Loss: {train_ciou:.6f}, mAP: {train_map:.2f}", end="")
        val_loss, val_ciou, valid_map = 0, 0, 0

        # Validation
        model.eval()
        with torch.no_grad():
            for curr_frame, prev_bbox, curr_bbox in valid_loader:
                prev_bbox, curr_bbox = prev_bbox.to(device), curr_bbox.to(device)
                _, pred_bbox = model(curr_frame.to(device), prev_bbox, use_teacher_forcing=True)  # Use Teacher Forcing while training

                val_loss += criterion(pred_bbox, curr_bbox).item() / BATCH_SIZE[0] / valid_length
                ciou_values = complete_box_iou(cxcywh_to_xyxy(pred_bbox), cxcywh_to_xyxy(curr_bbox))
                val_ciou += (1-torch.diag(ciou_values).mean()) / valid_length
                valid_progress.update(1)

        wandb.log({'Train MSE Loss': train_loss, 'Train CIoU Loss': train_ciou, 'Train mAP': train_map, 'Val MSE Loss': val_loss, 'Val CIoU Loss': val_ciou, 'Val mAP': valid_map})
        print(f"\rEpoch [{epoch+1:2}/{EPOCHS}], Step [{train_length}/{train_length}], MSE Loss: {train_loss:.6f}, CIoU Loss: {train_ciou:.6f}, mAP: {train_map:.2f}, Valid MSE Loss: {val_loss:.6f}, Valid CIoU Loss: {val_ciou:.6f}, Valid mAP: {valid_map:.2f}", end="\n" if (epoch+1) % 1 == 0 or (epoch+1) == EPOCHS else "")

        # Model Save
        target = model.module if ADDITIONAL_GPU else model
        save_path = path.join(".", "models", model_id, f"epoch{epoch}_{val_ciou:.6f}.pt")
        torch.save(target.apt.state_dict(), save_path)
        torch.save(target.state_dict(), save_path.replace(".pt", ".full.pt"))
        if best_ciou > val_ciou:
            best_ciou = val_ciou
            torch.save(target.apt.state_dict(), path.join(".", "models", model_id, f"best.pt"))

## Test

### APT Test

#### Test Dataset

In [ ]:
from collections import defaultdict

class TestVideoDataset(Dataset):
    def __init__(self, base_dataset: GOT10kDataset):
        super().__init__()
        self.base_dataset = base_dataset
        self.sequences = self._organize_sequences()

    def _organize_sequences(self):
        # Group images by sequence
        sequences = defaultdict(list)
        for idx, (img_path, _) in enumerate(self.base_dataset.samples):
            seq_name = path.dirname(img_path)
            sequences[seq_name].append((idx, img_path))

        # Sort frames in each sequence
        organized_sequences = []
        for seq_name, frames in sequences.items():
            # Sort frames by filename
            frames.sort(key=lambda x: x[1])

            # Get ground truth data
            gt_path = path.join(seq_name, 'groundtruth.txt')
            if path.exists(gt_path):
                groundtruth = np.loadtxt(gt_path, delimiter=',')

                # Get original image dimensions for normalization
                with Image.open(frames[0][1]) as img:
                    orig_w, orig_h = img.size

                # Normalize ground truth coordinates
                normalized_gt = groundtruth.copy()
                normalized_gt[:, 0] /= orig_w  # x_min
                normalized_gt[:, 1] /= orig_h  # y_min
                normalized_gt[:, 2] /= orig_w  # width
                normalized_gt[:, 3] /= orig_h  # height

                organized_sequences.append({
                    'frame_indices': [f[0] for f in frames],
                    'groundtruth': normalized_gt
                })

        return organized_sequences

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        sequence = self.sequences[idx]

        # Stack frames into a 4D tensor [frames, channels, height, width]
        frames = []
        for frame_idx in sequence['frame_indices']:
            img, _ = self.base_dataset[frame_idx]
            frames.append(img.unsqueeze(0))  # Add frame dimension

        frames_tensor = torch.cat(frames, dim=0)
        groundtruth = torch.FloatTensor(sequence['groundtruth'])

        return frames_tensor, groundtruth

In [ ]:
test_video_dataset = TestVideoDataset(base_dataset=valid_dataset)
test_video_loader = DataLoader(test_video_dataset, batch_size=1, shuffle=False, num_workers=cpu_cores)

#### Test Model

In [ ]:
test_length = len(test_video_loader)

model.eval()
for frames, groundtruth in tqdm(test_video_loader, desc="Testing Baseline..."):
    frames, groundtruth = frames.to(device), groundtruth.to(device)

    with torch.no_grad():
        output = model(frames)

    # Visualize output
    output = output.cpu().numpy()
    for j in range(len(output)):
        visualize_normalized_frame_pair(test_video_dataset, idx=i, figsize=(10, 5))
        print(f"Predicted Bounding Box: {output[j]}")
        print(f"Ground Truth Bounding Box: {groundtruth[j].cpu().numpy()}")

### Total Test

#### Test Dataset

In [ ]:
from apt.datasets import ImageNetVIDDataset
from tqdm import tqdm

In [ ]:
# Create transforms
eval_label_transform = lambda norm: transforms.Lambda(
    lambda x: ImageNetVIDDataset.label_transform(x, normalize=norm)
)
eval_test_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(**IMG_NORM)
])

In [ ]:
valid_dataset = ImageNetVIDDataset(
    root=DATA_ROOT, force_download=False, valid=True,
    transform=eval_label_transform, target_transform=eval_test_transform(norm=True)
)

print(f"INFO: Dataset loaded successfully. Number of samples - Eval Valid({len(valid_dataset)}))")

In [ ]:
for _ in range(2):
    selected_idx = valid_dataset.output_sampling(img_norm=IMG_NORM, imgsize=(IMG_SIZE, IMG_SIZE))
    print(f"Visualized pair index: {selected_idx}")

#### Baseline - Normal YOLOX

In [ ]:
# Load Model
yolo_baseline_id = "yoloxl_vid"

yolo_baseline = YOLOX(head=YOLOXHead(30))
yolo_baseline.load_state_dict(torch.load(f"./pretrained/{yolo_baseline_id}", map_location="cpu")['model'])
yolo_baseline.to(device)

In [ ]:
test_length = len(test_video_loader)

model.eval()
for prev_frame, curr_frame, prev_bbox, curr_bbox in tqdm(test_loader):
    prev_bbox, curr_bbox = prev_bbox.to(device), curr_bbox.to(device)
    _, pred_bbox, sparsity_loss = model(curr_frame.to(device), prev_bbox)

    val_loss += criterion(pred_bbox, curr_bbox).item() / valid_length
    val_ciou += (1-avg([calculate_ciou(pred_bbox[j].detach().cpu().numpy(), curr_bbox[j].cpu().numpy()) for j in range(len(pred_bbox))])) / valid_length
    valid_progress.update(1)

In [ ]:
# Load Model
our_model_id = "apt_model"

our_model = TestTimeAdaptiveYOLOX(
    encoder=yolo_baseline.backbone, decoder=yolo_baseline.head,
    feature_dim=256, bbox_dim=4, hidden_dim=512
)
our_model.apt.load_state_dict(torch.load(path.join(".", "models", model_id, f"best.pt")))
our_model.to(device)

In [ ]:
test_length = len(test_loader)


model.eval()
with torch.no_grad():
    for curr_frame, prev_bbox, curr_bbox in valid_loader:
        prev_bbox, curr_bbox = prev_bbox.to(device), curr_bbox.to(device)
        _, pred_bbox, sparsity_loss = model(curr_frame.to(device), prev_bbox, use_teacher_forcing=True)  # Use Teacher Forcing while training

        val_loss += criterion(pred_bbox, curr_bbox).item() / valid_length
        val_ciou += (1-avg([calculate_ciou(pred_bbox[j].detach().cpu().numpy(), curr_bbox[j].cpu().numpy()) for j in range(len(pred_bbox))])) / valid_length
        valid_progress.update(1)

model.eval()
with torch.no_grad():
    for inputs, targets in tqdm(test_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        corrects += (preds == targets.data).sum()
        print(f"Model Accuracy: {corrects/test_length:%}", end="\r")